# Camo Worms Genetic Algorithm

## Imports

In [ ]:
# libraries 
import numpy as np
import imageio.v3 as iio
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.bezier as mbezier
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
import random
import cv2
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.visualization import plot_param_importances
import sqlite3

rng = np.random.default_rng()
Path = mpath.Path
mpl.rcParams['figure.dpi']= 72 #size of images

## Image Preparation

## Global Variables (Experiment with different values)

In [ ]:
# Global variables

POP_SIZE = 100
EPOCHS = 30 # MAX NUMBER OF EPOCHS
CLEW_SIZE = 30
MUTATION_RATE = 0.01
MUTATION_INTENSITY = 0.02
WEIGHTS = {'size': 1.0, 'color': 1.0, 'overlap': 2.5, 'straightness': 0.1, 'edge': 1.0}
PATIENCE = 5
MIN_DELTA = 0
IMAGE_DIR = 'images'
IMAGE_NAME='original'
MASK = [320, 560, 160, 880] # ymin ymax xmin xmax
TOURNAMENT_SIZE = 15

# 'mutation_rate': 0.01, 'mutation_intensity': 0.02, 'size': 1.0, 
# 'color': 1.0, 'overlap': 2.5, 'straightness': 0.1, 'edge': 1.0

In [ ]:
def crop (image, mask):
    h, w = np.shape(image)
    return image[max(mask[0],0):min(mask[1],h), max(mask[2],0):min(mask[3],w)]

def prep_image (imdir, imname, mask):
    print("Image name (shape) (intensity max, min, mean, std)\n")
    image = np.flipud(crop(iio.imread(imdir+'/'+imname+".png"), mask))
    print("{} {} ({}, {}, {}, {})".format(imname, np.shape(image), np.max(image), np.min(image), round(np.mean(image),1), round(np.std(image),1)))
    plt.imshow(image, vmin=0, vmax=255, cmap='gray', origin='lower') # use vmin and vmax to stop imshow from scaling
    plt.show()
    return image

image = prep_image(IMAGE_DIR, IMAGE_NAME, MASK)

## Camo Worm Class

In [ ]:
class Camo_Worm:
    def __init__(self, x, y, r, theta, deviation_r, deviation_gamma, width, colour):
        self.x = x
        self.y = y
        self.r = r
        self.theta = theta
        self.dr = deviation_r
        self.dgamma = deviation_gamma
        self.width = width
        self.colour = colour
        p0 = [self.x - self.r * np.cos(self.theta), self.y - self.r * np.sin(self.theta)]
        p2 = [self.x + self.r * np.cos(self.theta), self.y + self.r * np.sin(self.theta)]
        p1 = [self.x + self.dr * np.cos(self.theta+self.dgamma), self.y + self.dr * np.sin(self.theta+self.dgamma)]
        self.bezier = mbezier.BezierSegment(np.array([p0, p1,p2]))

    def control_points(self):
        return self.bezier.control_points

    def path(self):
        return mpath.Path(self.control_points(), [Path.MOVETO, Path.CURVE3, Path.CURVE3])

    def patch(self):
        return mpatches.PathPatch(self.path(), fc='None', ec=str(self.colour), lw=self.width, capstyle='round')

    def intermediate_points(self, intervals=None):
        if intervals is None:
            intervals = max(3, int(np.ceil(self.r/8)))
        return self.bezier.point_at_t(np.linspace(0,1,intervals))

    def approx_length(self):
        intermediates = intermediate_points(self)
        eds = euclidean_distances(intermediates,intermediates)
        return np.sum(np.diag(eds,1))

## Draw Worms Class

In [ ]:
class Drawing:
    def __init__ (self, image):
        self.image = image

    def add_patches(self, patches):
        try:
            for patch in patches:
                self.ax.add_patch(patch)
        except TypeError:
            self.ax.add_patch(patches)

    def add_dots(self, points, radius=4, **kwargs):
        try:
            for point in points:
                self.ax.add_patch(mpatches.Circle((point[0],point[1]), radius, **kwargs))
        except TypeError:
            self.ax.add_patch(mpatches.Circle((points[0],points[1]), radius, **kwargs))

    def add_worms(self, worms, display=False):
        fig, ax = plt.subplots(figsize=(self.image.shape[1] / 100, self.image.shape[0] / 100), dpi=100)
        ax.imshow(self.image, cmap='gray', origin='upper')
        for worm in worms:
            ax.add_patch(worm.patch())
        ax.set_xlim([0, self.image.shape[1]])
        ax.set_ylim([0, self.image.shape[0]])
        ax.axis('off')
        if display:
            plt.show()
        fig.canvas.draw()
        data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)
        return data


    def show(self, save=None):
        if save is not None:
            plt.savefig(save)
        plt.show()

## Instance Class

In [ ]:
class Instance:
    def __init__(self, image):
        """Initialize an instance of the instance class with the image"""
        self.image = image
        self.clew = []
        self.cost = None
        self.drawing = None
        self.rng = np.random.default_rng()
        self.edges, self.distance_transform = self.prepare_edge_data()

    def detect_edges(self, image):
        """
        Detects edges using the Canny edge algo, returning the edges detected based on the threshold selected
        This process should be run manually to identify the appropriate thresholds (theshold1 and threshold2)
        as well as the degree in which dilation is applied (np.ones((2, 2), np.uint8))
        """
        # check if image is colored (has shape/channel of 3) and convert it to grayscale.
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # check the image is the correct data type for OpenCV (uint8)
        if image.dtype != np.uint8:
            image = np.uint8(255 * image)

        # for improved edge detection, reduce noise by applying a guassian blur 
        blurred_image = cv2.GaussianBlur(image, (7, 7), 2)
        
        # perform edge detection with Canny algo; note: low and high threshold values controls the edge detection sensitivity
        edges = cv2.Canny(blurred_image, threshold1=100, threshold2=20)

        # prepare a kernel for dilation and dilate the edges. allowing for thicker edges if required
        kernel = np.ones((2, 2), np.uint8)
        dilated_edges = cv2.dilate(edges, kernel, iterations=1)
        return dilated_edges
    
    def prepare_edge_data(self):
        """
        Process the image to detect edges and then computes a distance transform.
        The distance transform creates a area where each cell contains the distance to the nearest edge.  
        Allowing for the optimisation the worm location relative to edges detected with cv2
        """
        edges = self.detect_edges(self.image) # detect edges in the image using the method detect_edges
        # DIST_L2 allows for Euclidean distance to be used
        # DIST_MASK_PRECISE allows for more precise distance calc
        distance_transform = cv2.distanceTransform(255 - edges, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
        return edges, distance_transform # return edge data + distance transform

    def get_local_edges(self, x, y, radius):
        """Gets a local area around the given point for edge analysis."""
        # check that all values are set to integers and within image boundaries
        x1 = int(max(0, x - radius)) # left boundary
        y1 = int(max(0, y - radius)) # upper boundary
        x2 = int(min(self.image.shape[1], x + radius)) # right boundary
        y2 = int(min(self.image.shape[0], y + radius)) # lower boundary
        return self.edges[y1:y2, x1:x2]

    def calculate_size_penalty(self):
        """Calculates a size penalty based on worm parameters, proximity to edges, and deviations from the ideal size"""
        penalty = 0 # set penalty value
        
        # set ideal ratios for the length and width of a worm relative to the image size
        ideal_length_ratio, ideal_width_ratio = 0.7, 0.2

        # iterate over each worm in the clew
        for worm in self.clew:
            # ensure x and y values are within image boundaries; else out of image errors occurred
            x_idx = int(np.clip(worm.x, 0, self.image.shape[1] - 1))
            y_idx = int(np.clip(worm.y, 0, self.image.shape[0] - 1))

            # get the nearest edge distance from the distance transform previously calculated 
            nearest_edge_distance = self.distance_transform[y_idx, x_idx]

            # change the ideal length and width based on the worms distance to the nearest edge
            adjusted_ideal_length = ideal_length_ratio * self.image.shape[1] * (1 / (1 + nearest_edge_distance / 100))
            adjusted_ideal_width = ideal_width_ratio * self.image.shape[0] * (1 / (1 + nearest_edge_distance / 100))

            # calc the penalties for deviation from the adjusted ideal length and width
            length_penalty = (worm.r - adjusted_ideal_length) ** 2
            width_penalty = (worm.width - adjusted_ideal_width) ** 2

            # combine the penalties and apply a scaling factor; this shrinks the penalty for larger worms.
            combined_penalty = length_penalty + width_penalty
            scaled_penalty = combined_penalty * np.exp(-0.05 * worm.r)

            # sum the scaled penalty into the total penalty.
            penalty += scaled_penalty

        return penalty

    def calculate_ideal_straightness(self, worm):
        """Calculates the ideal straightness for a worm based on angle of Canny predicted edges"""
        # Check that x, y, and radius are integers to allow for indexing
        x = int(worm.x)
        y = int(worm.y)
        radius = int(worm.r + 20) # slightly larger radius to include a reasonable area around the worm

        # get edge data from the area surrounding the worm.
        local_edges = self.get_local_edges(x, y, radius)

        # calc gradients along the x and y directions with Sobel; helps in getting the direction of edges
        grad_x = cv2.Sobel(local_edges, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(local_edges, cv2.CV_64F, 0, 1, ksize=3)

        # calc the angle of the gradients to find the edge directions
        angle = np.arctan2(grad_y, grad_x)

        # make a hist of edge directions to find the most common edge position/orientation
        hist, _ = np.histogram(angle, bins=36, range=(-np.pi, np.pi))
        # find the bin with the max count, corresponding to the most prevalent edge orientation
        dominant_orientation = np.argmax(hist) * (2 * np.pi / 36) - np.pi # Convert bin index back to angle

        # Return the absolute value of the most common orientation as the ideal straightness measure
        return abs(dominant_orientation)

    def calculate_overlap_penalty(self):
        """Calculates penalty based on overlapping areas of worms to discourage worms overlapping one another"""

        # create a mask for the entire image with the same image dim but only using binary values
        mask = np.zeros_like(self.image, dtype=np.uint8)
        overlap_penalty = 0 

        # iterate over each worm in the clew
        for worm in self.clew:
            # make a new mask for the current worm.
            worm_mask = np.zeros_like(mask)

            # set the center, axes, and angle for the ellipse representing the worm
            center = (int(worm.x), int(worm.y)) # centre of the ellipse
            axes = (int(worm.width / 2), int(worm.r)) # axes of the ellipse; width and radius define the shape
            angle = np.degrees(worm.theta) # convert angle from radians to degrees

            # draw the worm as a filled ellipse on the worm-specific mask.
            cv2.ellipse(worm_mask, center, axes, angle, 0, 360, 255, -1)

            # get the overlap between the current worm and all other worms
            overlap = cv2.bitwise_and(mask, worm_mask)

            # if there is any overlap, try find the area of the overlap
            if np.any(overlap):
                overlap_area = np.sum(overlap > 0)
                overlap_penalty += overlap_area  # increase penalty based on overlap area

            # update the main mask with the current worm
            mask = cv2.bitwise_or(mask, worm_mask)  
            
        # Return the total overlap penalty (sum of all overlapping areas)
        return overlap_penalty

    def extract_local_area(self, x, y, radius):
        """Extracts a square around the worm"""

        # find the max size of the square without exceeding the image boundary
        half_side = int(min(radius * 1.5, min(x, y, self.image.shape[1] - x, self.image.shape[0] - y)))

        # calc the starting and ending x and y indices for the square region to be extracted.
        start_x = max(int(x - half_side), 0)
        end_x = min(int(x + half_side), self.image.shape[1])
        start_y = max(int(y - half_side), 0)
        end_y = min(int(y + half_side), self.image.shape[0])

        # return the subsection of the image
        return self.image[start_y:end_y, start_x:end_x]
    
    def add_random_worm(self, size, imshape, init_params):
        """Generates a list of worms with random values and adds them to the clew"""
        
        clew = [] # where generated worms are stored

        # get init params
        (radius_std, deviation_std, width_theta) = init_params
        (ylim, xlim) = imshape

        # gen each worm w/ rand values
        for _ in range(size):
            # randomly determine the center position of the worm within the image boundaries
            midx = xlim * self.rng.random() # x-coord
            midy = ylim * self.rng.random() # y-coord

            # randomly generate radius and deviation from the standard deviations 
            r = radius_std * np.abs(self.rng.standard_normal()) # radius
            dr = deviation_std * np.abs(self.rng.standard_normal()) # deviation from centre
             
            # randomly determine orientation and color deviation angles
            theta = self.rng.random() * np.pi  # orientation angle
            dgamma = self.rng.random() * np.pi # colour deviation angle
            colour = self.rng.random() # colour value

            # get the average colour of local area
            local_area = self.extract_local_area(midx, midy, r)
            if np.any(local_area):  # check if local_area contains any data
                local_colour = np.mean(local_area) / 255 # normalise and use as the local color

            # get the width of the worm using a gamma distribution scaled by a value
            width = width_theta * self.rng.standard_gamma(3)

            # create a new worm object with the randomly generated parameters.
            worm = Camo_Worm(midx, midy, r, theta, dr, dgamma, width, colour)
            clew.append(worm) # append worm to list (clew)

        return clew

    def calculate_cost(self):
        """Calculates the cost based on various penalties"""

        # calcs penalties, log-transformed and normalized by the clew size
        size_penalty = np.log1p(self.calculate_size_penalty()) / len(self.clew) 
        color_cost = 10*np.log1p(self.calculate_local_color_cost()) / len(self.clew)  
        overlap_penalty = np.log1p(self.calculate_overlap_penalty()) / len(self.clew)  
        straightness_penalty = np.log1p(sum(self.straightness_penalty(worm) for worm in self.clew)) / len(self.clew)  
        edge_cost = np.log1p(sum(self.edge_cost(self.detect_edges(self.image), worm) for worm in self.clew)) / len(self.clew)  

        # weight param, represents the importance of each cost component
        weights = WEIGHTS
        self.cost = (weights['size'] * size_penalty +
                    weights['color'] * color_cost +
                    weights['overlap'] * overlap_penalty +
                    weights['straightness'] * straightness_penalty +
                    weights['edge'] * edge_cost)

        return self.cost # returns the normalised total cost
    
    def add_clew(self, size, clews=None):
        """Adds the clew of worms to the instance and updates the drawing. It also calculates their cumulative cost"""
        
        # check if a pre-existing clew is present
        if clews:
            self.clew = clews  # if present, use the existing clew
        else: # if not, generate a new clew randomly
            self.clew = self.add_random_worm(size, image.shape, (40, 30, 1))

        # draw the worms on the image
        self.drawing = Drawing(self.image)
        # Add the clew to the drawing
        self.drawing.add_worms(self.clew)

        # calc the cost of the current clew of worms
        self.calculate_cost()

        # return the newly drawn clew of worms
        return self.drawing

    def show_instance(self):
        """Displays the current instance if available"""
        
        if self.drawing: # check if the drawing exists
            self.drawing.show() # if it exists display the image

    def calculate_local_color_cost(self):
        """Calculates cost based on color differences in localized regions"""
        total_color_distance = 0

        for worm in self.clew:
            # get the local image area around the worm
            local_area = self.extract_local_area(worm.x, worm.y, worm.r)

            if local_area.size > 0:
                # calc the average color intensity in the local area
                mean_local_color = np.mean(local_area) / 255

                # calc the absolute difference in color
                color_distance = abs(mean_local_color - worm.colour)

                # add this difference to the total color distance
                total_color_distance += color_distance
            else:
                # deal w/ cases where area is empty or not valid (assign a high penalty or default cost)
                total_color_distance += 1  # assuming max color distance is ~ 1

        return total_color_distance

    def edge_cost(self, edges, worm):
        """Calculates cost based on how closely the worm aligns with and covers edges"""
        # calc the distance transform of the edge image
        distance_transform = cv2.distanceTransform(edges, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)

        # make a binary mask for the worms path
        worm_mask = np.zeros_like(edges, dtype=np.uint8)

        # get the intermediate points along the Bezier curve that defines the worm's shape
        worm_points = worm.intermediate_points(intervals=100)  # intervals can be adjusted for a smoother line

        # draw points on the mask
        for point in worm_points:
            x, y = int(point[0]), int(point[1])
            if 0 <= x < worm_mask.shape[1] and 0 <= y < worm_mask.shape[0]:
                cv2.circle(worm_mask, (x, y), max(1, int(worm.width // 2)), 1, -1)  # draw circles to represent the worm's width

        # apply the worm mask to the distance transform to isolate distances at the worm's locations
        worm_distances = distance_transform * worm_mask

        # extract non-zero distances (where the worm is located)
        worm_edge_distances = worm_distances[worm_distances != 0]

        # calc the mean distance; lower distance means the worm is closer to an edge
        if worm_edge_distances.size > 0:
            mean_distance = np.mean(worm_edge_distances)
            # calc cost inversely proportional to the mean distance to encourage alignment with edges
            if mean_distance == 0:
                return 0  # no penalty if exactly covering the edge
            else:
                return 1 / mean_distance  # inverse relationship to encourage edge alignment
        else:
            # give a high penalty if the worm does not align close to any edges, discouraging this configuration.
            return 1000  # very high penalty to discourage placement away from edges
    
    def straightness_penalty(self, worm):
        """Calculates penalty based on deviation from straightness of closest edge, detected with detect_edges"""
        ideal_straightness = self.calculate_ideal_straightness(worm)
        return (worm.dgamma - ideal_straightness) ** 2

## Genetic Algorithm Class

In [ ]:
class Genetic_Algorithm:
    def __init__(self, mutation_rate=MUTATION_RATE, mutation_intensity=MUTATION_INTENSITY, patience=PATIENCE, min_delta=MIN_DELTA, tournament_size=TOURNAMENT_SIZE):
        self.mutation_rate = mutation_rate  # Default mutation rate is set to 10%
        self.mutation_intensity = mutation_intensity  # default is 0.05
        self.costs_per_epoch = []
        self.patience = patience  # epochs to wait for improvement
        self.min_delta = min_delta  # min improvement considered significant
        self.tournament_size = tournament_size

    def tournament_select(self, population): # tourney size is a hyperparam
        """Perform tournament select by choosing k (tournament-size) individuals from the population at random"""
        
        tournament = random.sample(population, self.tournament_size)
        # return the individual with the lowest cost from the tournament
        return min(tournament, key=lambda individual: individual.cost)

    def crossover(self, parent_1, parent_2):
        """Performs uniform crossover between two parents to produce a child"""
        
        child_cost = float('inf') # initialise childs cost

        # make a new set of 'clews' by randomly selecting each clew from clews of both parents
        new_clews = [random.choice([p1, p2]) for p1, p2 in zip(parent_1.clew, parent_2.clew)]

        # Create a new child 
        child = Instance(image) 
        child.add_clew(CLEW_SIZE, new_clews) # add the newly created clew to the child

        # assign the cost of the child 
        child_cost = child.cost
        return child # return the new child

    def mutation(self, instance):
        """Performs mutation on perc of population"""
        
        mutation_intensity = self.mutation_intensity  # consider lowering this if still too aggressive

        for worm in instance.clew:
            if random.random() < self.mutation_rate:
                # Slightly mutate color
                mutated_color = max(min(worm.colour + random.uniform(-mutation_intensity, mutation_intensity), 1), 0)
                worm.colour = mutated_color

            if random.random() < self.mutation_rate:
                # More controlled positional mutation
                position_change_factor = 0.1 * worm.r  # Smaller positional changes
                worm.x += random.uniform(-position_change_factor, position_change_factor)
                worm.y += random.uniform(-position_change_factor, position_change_factor)
            
                # Ensure worm coordinates are within image boundaries
                worm.x = np.clip(worm.x, 0, instance.image.shape[1] - 1)
                worm.y = np.clip(worm.y, 0, instance.image.shape[0] - 1)
            
                # More controlled rotation
                rotation_change = random.uniform(-0.05, 0.05) * np.pi
                worm.theta = (worm.theta + rotation_change) % (2 * np.pi)

            if random.random() < self.mutation_rate:
                # Controlled size mutation
                worm.r *= random.uniform(0.95, 1.5)  # Limit the growth/shrink factor
                worm.dr *= random.uniform(0.95, 1.5)

                # Controlled shape mutation
                dgamma_change = random.uniform(-0.1, 0.1) * np.pi
                worm.dgamma = (worm.dgamma + dgamma_change) % (2 * np.pi)

                # Width mutation within narrower bounds
                worm.width *= random.uniform(0.95, 1.05)

            # Update Bezier control points with new parameters
            worm.p0 = np.array([worm.x - worm.r * np.cos(worm.theta), worm.y - worm.r * np.sin(worm.theta)])
            worm.p2 = np.array([worm.x + worm.r * np.cos(worm.theta), worm.y + worm.r * np.sin(worm.theta)])
            worm.p1 = np.array([worm.x + worm.dr * np.cos(worm.theta + worm.dgamma),
                                worm.y + worm.dr * np.sin(worm.theta + worm.dgamma)])
            worm.bezier.contains_points = [worm.p0, worm.p1, worm.p2]

        instance.add_clew(CLEW_SIZE, instance.clew)
        return instance

    def display_individual(self, individual):
        """Draw the image with worms and display it"""
        
        individual.drawing.add_worms(individual.clew, display=True)
    
    def run_genetic_algorithm(self, population, epochs, display_interval=1):
        """Runs the genetic algorithm over a series of generations (epochs) to evolve the population to the lowest cost"""

        # initilise the best cost to infinity and a counter for early stopping
        best_cost = float('inf')
        no_improvement_count = 0  # counter to track epochs without significant improvement

        # iterate through each epoch
        for i in range(epochs):
            new_pop = []

            # create new pop by selecting parents and creating child
            while len(new_pop) < len(population):
                # select two parents via tournament selection
                parent_1 = self.tournament_select(population)
                parent_2 = self.tournament_select(population)

                # make a child through uniform crossover
                child = self.crossover(parent_1, parent_2)

                # mutate the child based on the mutation rate
                if random.random() < self.mutation_rate:
                    child = self.mutation(child)
                # add the new individual to the population
                new_pop.append(child)
                
            # swap the old population with the new one
            population = new_pop

            # calc the costs for new population and determine the best cost in this epoch
            costs = [individual.cost for individual in new_pop]
            epoch_best_cost = min(costs)

            # print performance at appropriate epoch intervals
            if (i + 1) % display_interval == 0:
                print(f'Epoch {i + 1}: Best cost = {epoch_best_cost}')
                print(f"Displaying best individual from Epoch {i + 1}")
                self.display_individual(population[costs.index(epoch_best_cost)])

            # apply early stopping 
            if best_cost - epoch_best_cost > self.min_delta:
                best_cost = epoch_best_cost
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            if no_improvement_count >= self.patience:
                print(f"Stopping early at epoch {i + 1}. Best cost hasn't improved significantly for {self.patience} epochs.")
                break

            # append the best cost of this epoch to track history
            self.costs_per_epoch.append(epoch_best_cost)

        # find the best individual after all epochs
        best_individual = min(population, key=lambda x: x.cost)
        print(f"Best individual cost: {best_individual.cost}")
        best_individual.show_instance()

        # Plot the history
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(1, len(self.costs_per_epoch) + 1), self.costs_per_epoch, marker='o')
        plt.title('Cost per Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Cost')
        plt.grid(True)
        plt.show()

        return best_individual



## Initialise Population and Run the Genetic Algorithm

In [ ]:
import warnings

warnings.filterwarnings('ignore')

def init_population(image, CLEW_SIZE=50):
    # Could potentially add logic to get more favourable population initialisation
    # For example manually adjusting worms to favourable positions??
    population = []

    for _ in range(POP_SIZE):

        instance = Instance(image)
        instance.add_clew(CLEW_SIZE)
        population.append(instance)

    return population

population = init_population(image)

ga = Genetic_Algorithm(mutation_rate=MUTATION_RATE, mutation_intensity=MUTATION_INTENSITY)

ga.run_genetic_algorithm(population, EPOCHS)

In [ ]:
from sklearn.model_selection import ParameterGrid

def run_genetic_algorithm_with_params(mutation_rate, mutation_intensity, population, EPOCHS):
    ga = Genetic_Algorithm(mutation_rate=mutation_rate, mutation_intensity=mutation_intensity)
    return ga.run_genetic_algorithm(population, EPOCHS)

param_grid = {
    'clew_size': [10, 20, 30, 40, 50],
    'mutation_rate': [0.01],
    'mutation_intensity': [0.01],
    'size': [1.0],
    'color': [1.0],
    'overlap': [2.5],
    'straightness': [0.01],
    'edge': [3.0]
}
    
grid = list(ParameterGrid(param_grid))

results = []

for params in grid:
    CLEW_SIZE = params['clew_size']
    WEIGHTS = {'size': params['size'], 'color': params['color'], 'overlap': params['overlap'], 'straightness': params['straightness'], 'edge': params['edge']}
    # Initialize population for each run to ensure fairness in comparisons
    population = init_population(image, CLEW_SIZE)  # Make sure the image is defined or passed appropriately
    
    # Run the genetic algorithm with the current parameters
    best_individual = run_genetic_algorithm_with_params(params['mutation_rate'], params['mutation_intensity'], population, EPOCHS)
    
    # Assume the run_genetic_algorithm function returns the best individual or some performance metric
    result = {
        'params': params,
        'cost': best_individual.cost  # Assuming the best individual has a cost attribute
    }
    results.append(result)
    print(f"Tested with {params}: Best Cost={best_individual.cost}")

# Find the best configuration
best_result = min(results, key=lambda x: x['cost'])
print(f"Best configuration: {best_result['params']} with Cost={best_result['cost']}")

In [ ]:
# Assuming 'results' is your list of dictionaries containing parameters and their respective costs.
best_result = min(results, key=lambda x: x['cost'])

# Display the best result
print("The best result is:")
print(f"Parameters: {best_result['params']}")
print(f"Cost: {best_result['cost']}")

In [ ]:
# get the best costs and corresponding clew sizes for plotting
costs = [r['cost'] for r in results]
clew_sizes = [r['params']['clew_size'] for r in results]

# plot the cost for each clew size
plt.figure(figsize=(10, 5))
plt.plot(clew_sizes, costs, marker='o', linestyle='', color='dodgerblue')
plt.xlabel('Clew Size')
plt.ylabel('Cost')
plt.title('Cost vs Clew Size')
plt.xticks(range(10, 51, 10))
plt.grid(True)

# save plot at png
plt.savefig('Cost_vs_Clew_Size.png')
plt.show()

In [ ]:
def detect_edges(image):
    # Check if image is already in grayscale, if not convert it
    if len(image.shape) == 3:  # This means image is colored (likely RGB or RGBA)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Ensure image is of type uint8
    if image.dtype != np.uint8:
        image = np.uint8(255 * image)  # Normalize back to range 0-255 if necessary
    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred_image = cv2.GaussianBlur(image, (7, 7), 2)
    # Use Canny edge detector
    # edges = cv2.Canny(blurred_image, threshold1=135, threshold2=20)
    edges = cv2.Canny(blurred_image, threshold1=100, threshold2=20)

    # dilate the edges so they merge with one another
    kernel = np.ones((3,3), np.uint8) # REMEMBER THIS IS SOMETHING TO ADJUST SEE BOT (7, 7)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)
    return dilated_edges

edges = detect_edges(image)

# Displaying the edges
plt.figure()
plt.imshow(edges, cmap='gray')
plt.colorbar()
plt.show()

The best result is:
Parameters: {'clew_size': 10, 'color': 0.5, 'edge': 1.0, 'mutation_intensity': 0.01, 'mutation_rate': 0.05, 'overlap': 5.0, 'size': 0.5, 'straightness': 0.0}
Cost: 0.045523603259346

Parameters: {'clew_size': 20, 'color': 0.5, 'edge': 1.0, 'mutation_intensity': 0.01, 'mutation_rate': 0.01, 'overlap': 2.5, 'size': 0.5, 'straightness': 0.01}
Cost: 1.332259864790154223

# Optuna

In [ ]:
import optuna

from sklearn.model_selection import ParameterGrid

def run_genetic_algorithm_with_params(mutation_rate, mutation_intensity, population, EPOCHS):
    ga = Genetic_Algorithm(mutation_rate=mutation_rate, mutation_intensity=mutation_intensity)
    return ga.run_genetic_algorithm(population, EPOCHS)
    
# study 3 was decent
# Create a study object that uses an SQLite database to store its state.
study = optuna.create_study(
    study_name='study_9',  # Unique identifier of the study.
    storage='sqlite:///study_9.db',  # Uses SQLite database stored as a .db file.
    direction='minimize',
    load_if_exists=True  # This will reload an existing study if the study_name matches.
)

def objective(trial):
    # Define the hyperparameters as trial suggestions
    clew_size = 50  # Fixed as per your grid
    mutation_rate = trial.suggest_categorical('mutation_rate', [0.01, 0.02])
    mutation_intensity = trial.suggest_categorical('mutation_intensity', [0.01, 0.02])
    size = trial.suggest_categorical('size', [0.5, 1.0, 1.5])
    color = trial.suggest_categorical('color', [0.5, 1.0])
    overlap = trial.suggest_categorical('overlap', [1.0, 2.5, 5.0])
    straightness = trial.suggest_categorical('straightness', [0.001, 0.01, 0.1])
    edge = trial.suggest_categorical('edge', [1.0, 3.0, 5.0])

    # Assuming you have a function to initialize the population
    population = init_population(image, clew_size)

    # Setting weights based on the suggested values
    weights = {'size': size, 'color': color, 'overlap': overlap, 'straightness': straightness, 'edge': edge}
    
    # Run the genetic algorithm
    best_individual = run_genetic_algorithm_with_params(mutation_rate, mutation_intensity, population, 50)  # EPOCHS fixed to 100 for example

    # Return the cost of the best individual as the objective to minimize
    return best_individual.cost

# Execute an optimization by using the above-defined objective function wrapped in the number of trials or using timeout
study.optimize(objective, n_trials=200)  # You can set n_trials or timeout

print("Best value: ", study.best_trial.value)
print("Best params: ", study.best_trial.params)

In [ ]:
# load in Optuna study
study = optuna.load_study(study_name='study_9', storage='sqlite:///study_9.db')

# print the best values 
print("Best value: ", study.best_trial.value)
print("Best params: ", study.best_trial.params)

In [ ]:
# eval hyperparameter importances as per Optuna YT guide
optuna.importance.get_param_importances(study)

# Plot the parameter importances
plot_param_importances(study)

In [ ]:
# plot a contour plot of the hyperparamsw
fig = optuna.visualization.plot_contour(study) 
fig.update_layout(height=1000)
fig.show()

In [ ]:
# get parameters sorted by the importance values
importances = optuna.importance.get_param_importances(study)
params_sorted = list(importances.keys())

# plot
fig = optuna.visualization.plot_rank(study, params=params_sorted)
fig.update_layout(height=1000)
fig.show()

# Grid Search for Tournament Size

In [ ]:
from sklearn.model_selection import ParameterGrid

def run_genetic_algorithm_with_params(mutation_rate, mutation_intensity, population, EPOCHS):
    ga = Genetic_Algorithm(mutation_rate=mutation_rate, mutation_intensity=mutation_intensity)
    return ga.run_genetic_algorithm(population, EPOCHS)

# Best params:  {'mutation_rate': 0.01, 'mutation_intensity': 0.02, 'size': 1.0, 'color': 1.0, 'overlap': 2.5, 
#'straightness': 0.1, 'edge': 1.0}
param_grid = {
    'clew_size': [50],
    'tournament_size': [5, 10, 15, 20, 25, 30],
    'mutation_rate': [0.01],
    'mutation_intensity': [0.02],
    'size': [1.0],
    'color': [1.0],
    'overlap': [2.5],
    'straightness': [0.1],
    'edge': [1.0]
}
    
grid = list(ParameterGrid(param_grid))

results = []

for params in grid:
    CLEW_SIZE = params['clew_size']
    TOURNAMENT_SIZE = params['tournament_size']
    
    WEIGHTS = {'size': params['size'], 'color': params['color'], 'overlap': params['overlap'], 'straightness': params['straightness'], 'edge': params['edge']}
    # Initialize population for each run to ensure fairness in comparisons
    population = init_population(image, CLEW_SIZE)  # Make sure the image is defined or passed appropriately
    
    # Run the genetic algorithm with the current parameters
    best_individual = run_genetic_algorithm_with_params(params['mutation_rate'], params['mutation_intensity'], population, EPOCHS)
    
    result = {
        'params': params,
        'cost': best_individual.cost  # get the cost value
    }
    results.append(result)
    print(f"Tested with {params}: Best Cost={best_individual.cost}")

# Find the best configuration
best_result = min(results, key=lambda x: x['cost'])

In [ ]:
# get the best costs and corresponding clew sizes for plotting
costs = [r['cost'] for r in results]
clew_sizes = [r['params']['tournament_size'] for r in results]

# plot the cost for each clew size
plt.figure(figsize=(10, 5))
plt.plot(clew_sizes, costs, marker='o', linestyle='', color='dodgerblue')
plt.xlabel('Tournament Size')
plt.ylabel('Cost')
plt.title('Cost vs Tournament Size')
plt.xticks(range(5, 35, 5))
plt.grid(True)

# save plot at png
plt.savefig('Cost_vs_Tournament_Size.png')
plt.show()

In [ ]:
import warnings

warnings.filterwarnings('ignore')
TOURNAMENT_SIZE = 15

def init_population(image, CLEW_SIZE=50):
    # Could potentially add logic to get more favourable population initialisation
    # For example manually adjusting worms to favourable positions??
    population = []

    for _ in range(POP_SIZE):

        instance = Instance(image)
        instance.add_clew(CLEW_SIZE)
        population.append(instance)

    return population

population = init_population(image)

ga = Genetic_Algorithm(mutation_rate=MUTATION_RATE, mutation_intensity=MUTATION_INTENSITY)

ga.run_genetic_algorithm(population, EPOCHS)

## Appendix - Different attempt to load in images

In [ ]:
# another approach to loading in images
# from PIL import Image
# different image prep method

# def crop(image, mask):
#     # Assuming mask is a tuple of (top, bottom, left, right) indices
#     h, w = image.shape
#     return image[max(mask[0], 0):min(mask[1], h), max(mask[2], 0):min(mask[3], w)]

# def prepare_image(filename, resize, mask=None):
#     # Load in image
#     image_path = f'images/{filename}'
#     image = Image.open(image_path)

#     # Resize the image to an appropriate size
#     image = image.resize(resize, Image.Resampling.LANCZOS)
    
#     # Convert the image to greyscale
#     greyscale_image = image.convert('L')
    
#     # Convert the greyscale image to a numpy array for analysis
#     image_array = np.array(greyscale_image)
    
#     # Apply cropping if a mask is provided
#     if mask:
#         image_array = crop(image_array, mask)
    
#     # Normalize the image array to be between 0 and 1 for processing
#     normalized_image_array = image_array / 255.0

#     # Flip the image vertically
#     normalized_image_array = np.flipud(normalized_image_array)

#     # Plotting the image
#     plt.imshow(normalized_image_array, cmap='gray')  # Use a grey colormap
#     plt.axis('off')  # Turn off axis numbers and ticks
#     plt.show()
    
#     return normalized_image_array

# # Mask format: (top, bottom, left, right)
# # mask is for cropping the image if required
# image = prepare_image('averaged.png', resize=(400, 200), mask=(50, 200, 0, 400))  